In [1]:
import numpy as np
import pandas as pd
import glob
import re

In [11]:
datadir = "../data/McGill-Billboard/"
# filepaths = glob.glob(datadir+'[0-9][0-9][0-9][0-9]/salami_chords.txt')
filepaths = glob.glob(datadir+'0688/salami_chords.txt')

In [12]:
fileidx = 0
filepath = filepaths[fileidx]

def ExtractFile(filepath):
    f = open(filepath, 'r')
    raw = f.readlines()
    f.close
    return raw

def ExtractMeta(raw):
    meta = ''.join(raw[:5])
    title = [x for x in re.split('# title: |\n', meta) if x and not x[0]=='#'][0]
    artist = [x for x in re.split('# artist: |\n', meta) if x and not x[0]=='#'][0]
    metre = [x for x in re.split('# metre: |\n', meta) if x and not x[0]=='#'][0]
    tonic = [x for x in re.split('# tonic: |\n', meta) if x and not x[0]=='#'][0]

    return (title, artist, metre, tonic)

raw = ExtractFile(filepath)
(title, artist, metre, tonic) = ExtractMeta(raw)

In [13]:
print((title, artist, metre, tonic))

('On The Road Again', 'Canned Heat', '4/4', 'E')


In [48]:
def ExtractPhrases(raw):
    raw = raw[6:-2]
    phrases = list()
    for row in raw:
        splitted = re.split('\t', row)[1:]
        if not splitted: continue
        splitted = splitted[0]
        
        splitted = re.split('\|', splitted)[1:]        
        if not splitted: continue
        
        phrase = splitted[:-1]

        # check for repeat
        if IsRepeat(splitted[-1][1:3]):
            phrase.append(splitted[-1][1:3])

        phrases.append(phrase)
    return phrases

def CleanPhrases(phrases):
    phrases_clean = list()
    for phrase in phrases:
        phrase_clean = [chord.strip() for chord in phrase]
        phrases_clean.append(phrase_clean)
    return phrases_clean
    
def FormatChords(phrases_clean):
    chords = list()
    
    for phrase in phrases_clean:
        # clean non-chords
        phrase_chord = [RemoveNoise(chord) for chord in phrase]

        # retrieve dot from previous chord
        phrase_chord = RetrieveDots(phrase_chord)

        # remove blank (noise-only) lines
        if all([not p for p in phrase_chord]): continue

        # check for repeat
        if IsRepeat(phrase_chord[-1]):
            repeated = phrase_chord[:-1]
            n_repeat = int(phrase_chord[-1][-1])
            phrase_chord = repeated*n_repeat

        chords.append(phrase_chord)
    # flatten 2d list
    chords = sum(chords, [])

    # split chords into bars
    chords = [chord.split() for chord in chords]

    return chords

def IsRepeat(str):
    if re.match(r'x\d', str):
        return True
    else:
        return False

def RemoveNoise(chord):
    chord_clean = re.sub(r'|\*|\(\d+/\d+\)|N|&pause', '', chord)
    chord_clean = chord_clean.split()
    chord_clean = ' '.join(chord_clean)
    return chord_clean

def RetrieveDots(phrase_chord):
    retrieved_phrase = list()
    for bar in phrase_chord:
        retrieved_bar = list()
        chords = bar.split()
        for c_i, chord in enumerate(chords):
            if chord=='.':
                retrieved_bar.append(chords[c_i-1])
            else:
                retrieved_bar.append(chord)
        retrieved_phrase.append(' '.join(retrieved_bar))

    return retrieved_phrase


phrases = ExtractPhrases(raw)
phrases_clean = CleanPhrases(phrases)
chords = FormatChords(phrases_clean)

this is ['E:1']
this is ['E:maj(11)']
this is ['E:7(#9)', 'E:7(#9)', 'E:7(#9) E:7(#9) G:1 A:1', 'E:7(#9)', 'x3']
this is ['E:7(#9)', 'E:7(#9)', 'E:maj(#9) E:maj(#9) G:maj A:maj', 'E:7(#9)', 'x3']
this is ['E:7(#9)', 'E:7(#9)', 'E:maj(#9) E:maj(#9) G:maj A:maj', 'E:7(#9)', 'x3']
this is ['E:7(#9)', 'E:7(#9)', 'E:maj(#9) E:maj(#9) G:maj A:maj', 'E:7(#9)', 'x3']
this is ['E:7(#9)', 'E:7(#9)', 'E:maj(#9) E:maj(#9) G:maj A:maj', 'E:7(#9)', 'x3']
this is ['E:7(#9)', 'E:7(#9)', 'E:maj(#9) E:maj(#9) G:maj A:maj', 'E:7(#9)', 'x3']
this is ['E:7(#9)', 'E:7(#9)', 'E:maj(#9) E:maj(#9) G:maj A:maj', 'E:7(#9)', 'x3']
this is ['E:7(#9)', 'E:7(#9)', 'E:maj(#9) E:maj(#9) G:maj A:maj', 'E:7(#9)', 'x1']
this is ['E:1']
this is ['E:maj(11)']


In [39]:
def CreateIndex(chords):
    indexs_bar = list()
    for i, bar in enumerate(chords):
        index_bar = [i]*len(bar)
        indexs_bar.append(index_bar)
    return indexs_bar
chords_index = CreateIndex(chords)
chords_index

[[0],
 [1],
 [2],
 [3],
 [4, 4, 4, 4],
 [5],
 [6],
 [7],
 [8, 8, 8, 8],
 [9],
 [10],
 [11],
 [12, 12, 12, 12],
 [13],
 [14],
 [15],
 [16, 16, 16, 16],
 [17],
 [18],
 [19],
 [20, 20, 20, 20],
 [21],
 [22],
 [23],
 [24, 24, 24, 24],
 [25],
 [26],
 [27],
 [28, 28, 28, 28],
 [29],
 [30],
 [31],
 [32, 32, 32, 32],
 [33],
 [34],
 [35],
 [36, 36, 36, 36],
 [37],
 [38],
 [39],
 [40, 40, 40, 40],
 [41],
 [42],
 [43],
 [44, 44, 44, 44],
 [45],
 [46],
 [47],
 [48, 48, 48, 48],
 [49],
 [50],
 [51],
 [52, 52, 52, 52],
 [53],
 [54],
 [55],
 [56, 56, 56, 56],
 [57],
 [58],
 [59],
 [60, 60, 60, 60],
 [61],
 [62],
 [63],
 [64, 64, 64, 64],
 [65],
 [66],
 [67],
 [68, 68, 68, 68],
 [69],
 [70],
 [71],
 [72, 72, 72, 72],
 [73],
 [74],
 [75],
 [76, 76, 76, 76],
 [77],
 [78],
 [79],
 [80, 80, 80, 80],
 [81],
 [82],
 [83],
 [84, 84, 84, 84],
 [85],
 [86],
 [87],
 [88, 88, 88, 88],
 [89],
 [90],
 [91]]

In [40]:
def Raw2Basic(chord_raw):
    root = chord_raw.split(':')[0]
    func = chord_raw.split(':')[1].split('/')[0]
    
    if func[:3]=='maj':
        func = 'maj'
    elif func[:3]=='min':
        func = 'min'
    elif func[:3]=='sus':
        func = func[:4]
    elif func[0]=='1' or func[0]=='5':
        func = '15'
    
    chord_basic = root+':'+func
    return chord_basic

def Raw2Root(chord_raw):
    root = chord_raw.split(':')[0]
    return root

In [184]:
def GetEnharmonic(note):
    if note == 'Db':
        return 'C#'
    elif note == 'Eb':
        return 'D#'
    elif note == 'E#':
        return 'F'
    elif note == 'Fb':
        return 'E'
    elif note == 'Gb':
        return 'F#'
    elif note == 'Ab':
        return 'G#'
    elif note == 'Bb':
        return 'A#'
    elif note == 'B#':
        return 'C'
    elif note == 'Cb':
        return 'B'
    return note

def Abs2RelRoot(chord_root, tonic):
    print(chord_root, tonic)
    chromatic = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    relative = ['I', 'I#', 'II', 'II#', 'III', 'IV', 'IV#', 'V', 'V#', 'VI', 'VI#', 'VII']

    tonic = GetEnharmonic(tonic)
    chord_root = GetEnharmonic(chord_root)

    tonic_pos = chromatic.index(tonic)
    chord_root_pos = chromatic.index(chord_root)

    dist = chord_root_pos - tonic_pos
    rel_chord_root = relative[dist]

    return rel_chord_root

def Abs2Rel(chord, tonic):
    splitted = chord.split(':')
    root = splitted[0]
    func = ''
    if len(splitted) > 1:
        func = ':' + splitted[1]
    rel_root = Abs2RelRoot(root, tonic)
    rel_chord = rel_root + func

    return rel_chord

Abs2Rel('Bb:maj7', 'Ab')

Bb Ab


'II:maj7'

In [185]:
def Rel2InCRoot(root_rel, tonic='C'):
    chromatic = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    relative = ['I', 'I#', 'II', 'II#', 'III', 'IV', 'IV#', 'V', 'V#', 'VI', 'VI#', 'VII']

    chromatic = chromatic[chromatic.index(tonic):] + chromatic[:chromatic.index(tonic)]
    root_inc = chromatic[relative.index(root_rel)]

    return root_inc

def Rel2InC(chord_rel, tonic='C'):
    splitted = chord_rel.split(':')
    root = splitted[0]
    func = ''
    if len(splitted) > 1:
        func = ':' + splitted[1]
    rel_root = Rel2InCRoot(root, tonic)
    rel_chord = rel_root + func

    return rel_chord

In [186]:
df_chords = pd.DataFrame()
df_chords['bar'] = sum(chords_index, [])
df_chords['chords'] = sum(chords, [])
df_chords['chords_basic'] = df_chords['chords'].apply(Raw2Basic)
df_chords['chords_root'] = df_chords['chords'].apply(Raw2Root)

df_chords['chords_rel'] = df_chords['chords'].apply(Abs2Rel, tonic=tonic)
df_chords['chords_basic_rel'] = df_chords['chords_basic'].apply(Abs2Rel, tonic=tonic)
df_chords['chords_root_rel'] = df_chords['chords_root'].apply(Abs2Rel, tonic=tonic)

df_chords['chords_inC'] = df_chords['chords_rel'].apply(Rel2InC)
df_chords['chords_basic_inC'] = df_chords['chords_basic_rel'].apply(Rel2InC)
df_chords['chords_root_inC'] = df_chords['chords_root_rel'].apply(Rel2InC)

df_chords['meta'] = ['title: '+title, 'artist: '+artist, 'metre: '+metre, 'tonic: '+tonic] + [np.nan] * (len(df_chords)-4)

D G
G G
G G
G G
G G
G G
D G
F G
C G
G G
E G
G G
C G
G G
G G
D G
F G
C G
G G
E G
G G
C G
G G
B G
B G
E G
A G
A G
D G
G G
C G
C G
C G
B G
D G
G G
D G
F G
C G
G G
E G
G G
D G
A G
A G
E G
G G
D G
A G
F# G
A G
D G
A G
C# G
C# G
F# G
B G
B G
E G
A G
D G
D G
D G
C# G
D G
D G
A G
A G
F# G
F G
G G
A G
A G
F# G
F G
G G
A G
A G
F# G
F G
G G
A G
A G
F# G
F G
G G
A G
D G
G G
G G
G G
G G
G G
D G
F G
C G
G G
E G
G G
C G
G G
G G
D G
F G
C G
G G
E G
G G
C G
G G
B G
B G
E G
A G
A G
D G
G G
C G
C G
C G
B G
D G
G G
D G
F G
C G
G G
E G
G G
D G
A G
A G
E G
G G
D G
A G
F# G
A G
D G
A G
C# G
C# G
F# G
B G
B G
E G
A G
D G
D G
D G
C# G
D G
D G
A G
A G
F# G
F G
G G
A G
A G
F# G
F G
G G
A G
A G
F# G
F G
G G
A G
A G
F# G
F G
G G
A G
D G
G G
G G
G G
G G
G G
D G
F G
C G
G G
E G
G G
C G
G G
G G
D G
F G
C G
G G
E G
G G
C G
G G
B G
B G
E G
A G
A G
D G
G G
C G
C G
C G
B G
D G
G G
D G
F G
C G
G G
E G
G G
D G
A G
A G
E G
G G
D G
A G
F# G
A G
D G
A G
C# G
C# G
F# G
B G
B G
E G
A G
D G
D G
D G
C# G
D G
D G
A G
A G
F# G
F G


In [143]:
np.array(df_chords['chords'])

array(['D:maj', 'G:maj', 'G:maj', 'G:maj', 'G:maj', 'G:maj', 'D:maj/3',
       'F:maj', 'C:maj', 'G:maj', 'E:min', 'G:maj7', 'C:maj7', 'G:maj',
       'G:maj', 'D:maj/3', 'F:maj', 'C:maj', 'G:maj', 'E:min', 'G:maj',
       'C:maj', 'G:maj', 'B:sus4(b7,9)', 'B:7', 'E:min7', 'A:sus4(b7)',
       'A:7', 'D:maj', 'G:maj', 'C:maj', 'C:maj7', 'C:maj6', 'B:min',
       'D:maj/b7', 'G:maj', 'D:maj/3', 'F:maj', 'C:maj', 'G:maj', 'E:min',
       'G:maj', 'D:maj7', 'A:maj', 'A:maj', 'E:maj/3', 'G:maj', 'D:maj',
       'A:maj', 'F#:min', 'A:maj', 'D:maj', 'A:maj', 'C#:sus4(b7,9)',
       'C#:7', 'F#:min7', 'B:sus4(b7)', 'B:7', 'E:maj', 'A:maj', 'D:maj7',
       'D:maj6', 'D:maj6', 'C#:min7', 'D:maj13', 'D:maj13', 'A:maj',
       'A:maj/b7', 'F#:min', 'F:maj', 'G:maj', 'A:maj', 'A:maj/b7',
       'F#:min', 'F:maj', 'G:maj', 'A:maj', 'A:maj/b7', 'F#:min', 'F:maj',
       'G:maj', 'A:maj', 'A:maj/b7', 'F#:min', 'F:maj', 'G:maj', 'A:maj'],
      dtype=object)

In [115]:
csvpath = '/'.join(filepath.split('/')[:-1]) + '/chords.csv'
df_chords.to_csv(csvpath)

In [158]:
print(tonic)